# Выбор локации для скважины

Задача:
На основе данных геологи разведки выбрать район добычи нефти

## Загрузка и подготовка данных

### Первичный анализ

In [ ]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import math
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, mean_squared_error, roc_auc_score
from sklearn.utils import shuffle
from numpy.random import RandomState

In [ ]:
# Загружаем данные
try:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_data_0 = pd.read_csv('C:/Users/Student/Desktop/geo_data_0.csv')
    geo_data_1 = pd.read_csv('C:/Users/Student/Desktop/geo_data_1.csv')
    geo_data_2 = pd.read_csv('C:/Users/Student/Desktop/geo_data_2.csv')

In [ ]:
# Изучаем данные
geo_data_0.info()
geo_data_1.info()
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Пропусков в данных нет, типы данных обозначены правильно

In [ ]:
# Первые 5 строчек
print(geo_data_0.head())
print(geo_data_1.head())
geo_data_2.head()

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
# Смотрим основную описательную статистику
print(geo_data_0.describe())
print(geo_data_1.describe())
geo_data_2.describe()

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Все средние примерно равны медианам, значит признаки нескошены. Стандартные отклонения разнятся незначительно, значит масштабирование необязательно.

In [ ]:
# Исследуем проблему мультиколлинеарности
print(geo_data_0.corr())
print(geo_data_1.corr())
geo_data_2.corr()

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Проблема мультиколиниарности между признаками нигде не встречается, с данными всё в порядке. Мы заметили высокую корреляцию между признаком f2 и таргетом в geo_1. Возможно это признак близости к основному местурождению сырья, поэтому чем дальше от него (чем меньше значение), тем меньше объём сырья.

### Деление на выборки

In [ ]:
# Делим на целевой и остальные признаки
geo_data_0_features = geo_data_0.drop(['id', 'product'], axis=1)
geo_data_1_features = geo_data_1.drop(['id', 'product'], axis=1)
geo_data_2_features = geo_data_2.drop(['id', 'product'], axis=1)

geo_data_0_target = geo_data_0['product']
geo_data_1_target = geo_data_1['product']
geo_data_2_target = geo_data_2['product']

In [ ]:
# Делим все данные на тренировочные и тестовую выборки
geo_data_0_features_train, geo_data_0_features_valid, geo_data_0_target_train, geo_data_0_target_valid = train_test_split(
    geo_data_0_features, geo_data_0_target, test_size=0.25, random_state=12345
    )

geo_data_1_features_train, geo_data_1_features_valid, geo_data_1_target_train, geo_data_1_target_valid = train_test_split(
    geo_data_1_features, geo_data_1_target, test_size=0.25, random_state=12345
    )

geo_data_2_features_train, geo_data_2_features_valid, geo_data_2_target_train, geo_data_2_target_valid = train_test_split(
    geo_data_2_features, geo_data_2_target, test_size=0.25, random_state=12345
    )

# Сколько объектов осталось в выборках
print(geo_data_0_features_train.shape[0])
print(geo_data_0_features_valid.shape[0])

75000
25000


### Подвывод

Мы загрузили нужные библиотеки, данные, провели первичный анализ. Пропусков в данных нет, проблемы мултиколлиниарности тоже нет. Данные не требовали масштабирования, скошенности в данных нет. Мы разделили данные по каждому региону на целевые и остальные признаки, каждые в свою очередь разделили на тренировочные и валидационные выборки.

## Обучение и проверка модели

По условию поставленной задачи мы будем использовать линейную регрессию (наш таргет -- численный), качество будем проверять на RMSE метрике

### ML для geo_0 региона

In [ ]:
model = LinearRegression()
model.fit(geo_data_0_features_train, geo_data_0_target_train)
predictions_geo_0 = model.predict(geo_data_0_features_valid)
mse = mean_squared_error(geo_data_0_target_valid, predictions_geo_0)
rmse = mse ** 0.5
print('RMSE:', rmse, '|',
      'Среднее предсказаний:', predictions_geo_0.mean(), '|',
      'Среднее изначальных данных', geo_data_0_target_valid.mean())

RMSE: 37.5794217150813 | Среднее предсказаний: 92.59256778438035 | Среднее изначальных данных 92.07859674082927


### ML для geo_1 региона

In [ ]:
model = LinearRegression()
model.fit(geo_data_1_features_train, geo_data_1_target_train)
predictions_geo_1 = model.predict(geo_data_1_features_valid)
mse = mean_squared_error(geo_data_1_target_valid, predictions_geo_1)
rmse = mse ** 0.5
print('RMSE:', rmse, '|',
      'Среднее предсказаний:', predictions_geo_1.mean(), '|',
      'Среднее изначальных данных', geo_data_1_target_valid.mean())

RMSE: 0.893099286775617 | Среднее предсказаний: 68.728546895446 | Среднее изначальных данных 68.72313602435997


### ML для geo_2 региона

In [ ]:
model = LinearRegression()
model.fit(geo_data_2_features_train, geo_data_2_target_train)
predictions_geo_2 = model.predict(geo_data_2_features_valid)
mse = mean_squared_error(geo_data_2_target_valid, predictions_geo_2)
rmse = mse ** 0.5
print('RMSE:', rmse, '|',
      'Среднее предсказаний:', predictions_geo_2.mean(), '|',
      'Среднее изначальных данных', geo_data_2_target_valid.mean())

RMSE: 40.02970873393434 | Среднее предсказаний: 94.96504596800489 | Среднее изначальных данных 94.88423280885438


### Подвывод

Самое маленькое RMSE получилось при предсказании объёмов сырья в регионе geo_1, так как там был признак с высокой (0.999) корреляцией с таргетом, поэтому разница между предсказанными и изначальными объёмами разнится на 0.1 тыс. баррелей. В остальных регионах предсказания также не сильно отличаются от изначальных объёмов, разница составляет также ~0.1 тыс. баррелей для остальных регионов.

## Подготовка к расчёту прибыли

### Создание новых переменных

In [ ]:

product_0 = pd.Series(predictions_geo_0)
product_1 = pd.Series(predictions_geo_1)
product_2 = pd.Series(predictions_geo_2)
product_0

0         95.894952
1         77.572583
2         77.892640
3         90.175134
4         70.510088
            ...    
24995    103.037104
24996     85.403255
24997     61.509833
24998    118.180397
24999    118.169392
Length: 25000, dtype: float64

In [ ]:
# Бюджет на разработку скважин в регионе
TOTAL_BUDGET = 10_000_000_000
# Стоимость одной единицы product
ONE_PRODUCT_PRICE = 450_000
WELL_CAPACITY = 200

### Рассчёт достаточного объёма сырья для безубыточной разработки новой скважины

In [ ]:
# Сколько надо продукта для точки безубыточности
print('Сколько надо продукта для точки безубыточности', math.ceil(TOTAL_BUDGET / ONE_PRODUCT_PRICE))
# Среднее кол-во баррелей в каждой из 200 лучших скважин для достижения точки безубыточности
print('Среднее кол-во баррелей в каждой из 200 лучших', math.ceil(TOTAL_BUDGET / ONE_PRODUCT_PRICE)/WELL_CAPACITY)
# Средний объём сырья для региона geo_0
print('Средний объём сырья для региона geo_0', geo_data_0_target.mean())
# Средний объём сырья для региона geo_1
print('Средний объём сырья для региона geo_1',geo_data_1_target.mean())
# Средний объём сырья для региона geo_2
print('Средний объём сырья для региона geo_2',geo_data_2_target.mean())

Сколько надо продукта для точки безубыточности 22223
Среднее кол-во баррелей в каждой из 200 лучших 111.115
Средний объём сырья для региона geo_0 92.50000000000001
Средний объём сырья для региона geo_1 68.82500000000002
Средний объём сырья для региона geo_2 95.00000000000004


### Подвывод

Точка безубыточности достигается в точке 22223 проданных единиц product, иными словами в среднем в каждой из 200 выборанных скаважин в среднем должно быть 111.115 тыс. баррелей. В то же время, средний объём сырья в каждом регионе сильно меньше необходимого нам порога:

 - geo_0 = 92.5 тыс. баррелей
 - geo_1 = 68.8 тыс. баррелей
 - geo_2 = 95 тыс. баррелей

## Расчёт прибыли и рисков

### Функция прибыли

In [ ]:
# Функция для подсчёта прибыли
def profit(pred_data, prod_data, n_wells, price_for_one_product, cost):
    pred_data_sorted_index = pred_data.sort_values(ascending=False).head(n_wells).index
    product_data = prod_data.reset_index(drop=True)[pred_data_sorted_index]
    sum_income = sum(product_data * price_for_one_product)
    return sum_income - cost


In [ ]:
# Подсчёт прибыли для каждого региона на основе предсказаний
profit_geo_0 = profit(product_0, geo_data_0_target_valid, WELL_CAPACITY, ONE_PRODUCT_PRICE, TOTAL_BUDGET)
profit_geo_1 = profit(product_1, geo_data_1_target_valid, WELL_CAPACITY, ONE_PRODUCT_PRICE, TOTAL_BUDGET)
profit_geo_2 = profit(product_2, geo_data_2_target_valid, WELL_CAPACITY, ONE_PRODUCT_PRICE, TOTAL_BUDGET)
print("Прибыль для geo_0", profit_geo_0, '|', "Прибыль для geo_1", profit_geo_1,'|', "Прибыль для geo_2", profit_geo_2)

Прибыль для geo_0 3320826043.1398487 | Прибыль для geo_1 2415086696.6815624 | Прибыль для geo_2 2710349963.599825


## Bootstrap

### Функция подсчёта риска убытка

In [ ]:
# Функция подсчёта риска убытка
def risk(data):
    count = 0
    for i in range(len(values)):
        if values[i] < 0:
            count += 1

    return count/len(data)

### Распределение прибыли

#### geo_0

In [ ]:
state = RandomState(12345)
values = []
best_mean = 0
for i in range(1000):
    pred_subsample = product_0.sample(n=500, replace=True, random_state=state)
    values.append(profit(pred_subsample, geo_data_0_target_valid, WELL_CAPACITY, ONE_PRODUCT_PRICE, TOTAL_BUDGET))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()

if best_mean < mean:
    best_mean = mean

print("Средняя прибыль:", mean)
print("95% доверительный интервал:", lower, upper)

Средняя прибыль: 396164984.80237025
95% доверительный интервал: -111215545.89049712 909766941.5534189


In [ ]:
# Вероятность убытка
risk(values)

0.069

#### geo_1

In [ ]:
values = []
for i in range(1000):
    pred_subsample = product_1.sample(n=500, replace=True, random_state=state)
    values.append(profit(pred_subsample, geo_data_1_target_valid, WELL_CAPACITY, ONE_PRODUCT_PRICE, TOTAL_BUDGET))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)


mean = values.mean()

if best_mean < mean:
    best_mean = mean

print("Средняя прибыль:", mean)
print("95% доверительный интервал:", lower, upper)


Средняя прибыль: 461155817.27721286
95% доверительный интервал: 78050810.75172454 862952060.2636913


In [ ]:
# Вероятность убытка
risk(values)

0.007

#### geo_2

In [ ]:
values = []
for i in range(1000):
    pred_subsample = product_2.sample(n=500, replace=True, random_state=state)
    values.append(profit(pred_subsample, geo_data_2_target_valid, WELL_CAPACITY, ONE_PRODUCT_PRICE, TOTAL_BUDGET))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()

if best_mean < mean:
    best_mean = mean

print("Средняя прибыль:", mean)
print("95% доверительный интервал:", lower, upper)

Средняя прибыль: 392950475.17060584
95% доверительный интервал: -112227625.3785718 934562914.551158


In [ ]:
# Вероятность убытка
risk(values)

0.065

### Лучшая прибыль

In [ ]:
#Наибольшая прибыль
best_mean

461155817.27721286

### Подвывод

Среди всех регионов, наибольшая средняя прибыль будет у региона geo_1 и будет равняться ~ 360 584 036 рублей, что сильно отличается от средней прибыли в других регионах (294 905 854 и 291 677 673).

Вероятность убытков в каждом регионе:
 - geo_0 = 6.9%
 - geo_1 =  0.7%
 - geo_2 =  6.5%

Добыча в каждом регионе с 95% вероятностью попадёт в диапазон:
 - geo_0 [-220057306 ,   813659028]
 - geo_1 [-36074998  ,  762306510]
 - geo_2 [-203487531 ,   826093983]

## Вывод

В ходе работы мы провели первичный анализ данных, поделили все данные на выборки, обучили модель и предсказали с высокой точностью объём продукта. Далее мы создали функцию подсчёта прибыли и риска убытков, рассчитали среднее достаточное кол-во ресурса для преодоления безубыточности. В процессе Bootstrap мы подсчитали 95% доверительный интервал прибыли из 500 случайно выбранных скважин, 200 из которых мы взяли для добычи. В результате риск убытков для регионов geo_0 и geo_2 равняется примерно 7%, а geo_1 меньше 1%. <b>Самым прибыльным и самым стабильным из 3-х регионов оказался geo_1 с прогнозируемой прибылью ~  360 584 036 рублей, его и рекомендуется выбрать для выработки</b>.